In [5]:
from optimization import * 
from Kernels import * 


def fit(X_train,X_test, y):
    global w,a_org,b
    kernel = gaussian_kernel
    n_samples, n_features = X_train.shape 
    nt_samples, nt_features = X_test.shape
    # Gram matrix

    print(K.shape)

    P = cvxopt.matrix(np.outer(y,y) * K)
    q = cvxopt.matrix(np.ones(n_samples) * -1)
    A = cvxopt.matrix(y, (1,n_samples))
    A = matrix(A, (1,n_samples), 'd') #changes done
    b = cvxopt.matrix(0.0)
    #print(P,q,A,b)
    if C is None:
        G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h = cvxopt.matrix(np.zeros(n_samples))

    else:
        tmp1 = np.diag(np.ones(n_samples) * -1)
        tmp2 = np.identity(n_samples)
        G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
        tmp1 = np.zeros(n_samples)
        tmp2 = np.ones(n_samples) * C
        h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
    # solve QP problem
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    print(solution['status'])
    # Lagrange multipliers
    a = np.ravel(solution['x'])
    a_org = np.ravel(solution['x'])
    # Support vectors have non zero lagrange multipliers
    for i in range(n_samples):
        sv_=np.logical_or(a_org <m, a_org > 1e-5)
    #print(sv.shape)
    ind = np.arange(len(a))[sv_]
    a = a[sv_]
    sv = X_train[sv_]
    sv_y = y[sv_]

    # Intercept
    b = 0
    for n in range(len(a)):
        b += sv_y[n]
        b -= np.sum(a * sv_y * K[ind[n],sv_])
    b /= len(a)
    print(b)

    # Weight vector
    if kernel == gaussian_kernel:
        w = np.zeros(n_features)
        for n in range(len(a)):
            w += a[n] * sv_y[n] * sv[n]
    else :
        w = None        

def project(X):
    if w is None:
        return np.dot(X, w) + b
    else:
        y_predict = np.zeros(len(X))
        X=np.asarray(X)
        for i in range(len(X)):
            s = 0
            for a_, sv_y_, sv_ in zip(a, sv_y, sv):
                s += a_ * sv_y_ * gaussian_kernel(X[i], sv_)
            y_predict[i] = s
          #  print(y_predict[i])
        return y_predict + b

def predict( X):
    return np.sign(project(X))




train = pd.read_csv("data/modifiedpima.csv", header=None)
train.head()

features = train.columns[0:8]
X = train[features]
y = train[8]
X.head()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)
print(X_train.shape,X_test.shape)
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)

if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        init_cst(C_=100.0)
        typ=2
        m_func(X_train,X_test,y_train)
        fit(X_train,X_test, y_train)
        y_predict = predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    hyp_svm()

(614, 8) (154, 8)
(614, 614)
     pcost       dcost       gap    pres   dres
 0:  3.1997e+04 -2.6998e+06  3e+06  1e-01  3e-13
 1:  7.0493e+02 -3.7645e+05  4e+05  1e-02  2e-13
 2: -1.5080e+04 -8.7294e+04  7e+04  2e-03  2e-13
 3: -2.0698e+04 -4.2280e+04  2e+04  3e-04  3e-13
 4: -2.3448e+04 -3.5410e+04  1e+04  2e-04  3e-13
 5: -2.4919e+04 -3.1494e+04  7e+03  7e-05  3e-13
 6: -2.5867e+04 -2.9015e+04  3e+03  3e-05  4e-13
 7: -2.6304e+04 -2.8026e+04  2e+03  1e-05  4e-13
 8: -2.6625e+04 -2.7314e+04  7e+02  2e-06  4e-13
 9: -2.6775e+04 -2.7035e+04  3e+02  7e-07  4e-13
10: -2.6855e+04 -2.6900e+04  4e+01  4e-08  4e-13
11: -2.6872e+04 -2.6874e+04  2e+00  1e-09  4e-13
12: -2.6873e+04 -2.6873e+04  4e-02  2e-11  4e-13
13: -2.6873e+04 -2.6873e+04  6e-04  4e-13  5e-13
Optimal solution found.
optimal
(268,)
(614,)
(614, 614)
     pcost       dcost       gap    pres   dres
 0:  3.1997e+04 -2.6998e+06  3e+06  1e-01  3e-13
 1:  7.0493e+02 -3.7645e+05  4e+05  1e-02  2e-13
 2: -1.5080e+04 -8.7294e+04  7e+04